$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Introduction aux réseaux de neurones : TD #2  (partie 2)
Matériel de cours rédigé par Pascal Germain, 2018
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn

### Chargement des données «MNIST»

In [ ]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire_mnist + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [ ]:
repertoire_mnist = 'mnist/' # Modifier le répertoire au besoin
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=None, max_par_etiquettes=1000)
data_x = data_x / 255

In [ ]:
print('data_x:', data_x.shape)
print('data_y:', data_y.shape)

In [ ]:
plt.figure(figsize=(15,4))
plt.imshow(data_x, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

Nous sélectionnons aléatoirement 4 images de l'ensemble MNIST qui servirons à illustrer les couches convolutives

In [ ]:
nb_images = 4
dimension = 28
indices = np.random.randint(len(data_y), size=nb_images)
sample_x = data_x[indices,:]
sample_x.shape

Convertissons ces images en **tenseurs pyTorch** de 4 dimensions: $m \times c \times h \times l$, où:
1. $m$ est le nombre d'images (lors de l'apprentissage, cela correspondra à la taille de la «minibatch»
2. $c$ est le nombre de «canaux» de l'image. Ici, nous avons un seul canal, car les images sont en teinte de gris. Typiquement, une image  couleur aura trois canaux: rouge, vert, bleu.
3. $h$ correspond à la hauteur de l'image, en nombre de pixels.
4. $c$ correspond à la largeur de l'image, en nombre de pixels.

In [ ]:
images = torch.tensor(sample_x).view(nb_images, 1, dimension, dimension)
images.shape

Le module `torchvision` contient des fonctions utilitaires pour le traitement des images. Ici, nous utiliserons la fonction `make_grid` pour afficher plusieurs images à la fois

In [ ]:
from torchvision.utils import make_grid
def afficher_grille(images):
    plt.figure(figsize=(15,4))
    grid = make_grid(images, pad_value=torch.max(images))
    plt.imshow(grid[0].detach(), cmap=plt.cm.gray)
    plt.colorbar()

In [ ]:
afficher_grille(images)

### Convolutions

In [ ]:
nn.Conv2d?

Nous nous concentrerons sur les 3 premiers arguments nécessaires pour la création d'un filtre convolutif:
1. Le nombre de canaux (ici 1, car notre image est en teintes de gris. Ce serait 3 pour une image en couleur),
2. Le nombre de canaux de sortie. Autrement dit, le nombre de filtres apprit (1 pour l'exemple ci-bas, mais on utilise typiquement une valeur plus élevée dans un réseau de neurones).
3. La taille de chaque filtre (en nombre de pixels).

In [ ]:
conv = nn.Conv2d(1,1,3)

In [ ]:
conv.weight

In [ ]:
conv.weight.shape

In [ ]:
conv.bias

In [ ]:
conv_images = conv(images)

In [ ]:
conv_images.shape

In [ ]:
afficher_grille(conv_images)

Essayons quelques filtres de Sobel.
Voir https://fr.wikipedia.org/wiki/Filtre_de_Sobel

In [ ]:
left_sobel =  nn.Conv2d(1,1,3, bias=False)
left_sobel.weight = nn.Parameter(torch.Tensor([[[[1, 0, -1], [2, 0, -2], [1, 0, -1]]]]))
afficher_grille(left_sobel(images))

In [ ]:
left_sobel.weight

In [ ]:
right_sobel = nn.Conv2d(1,1,3, bias=False)
right_sobel.weight = nn.Parameter(torch.Tensor([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]]))
afficher_grille(right_sobel(images))

In [ ]:
up_sobel =  nn.Conv2d(1,1,3, bias=False)
up_sobel.weight = nn.Parameter(torch.Tensor([[[[1,2,1], [0, 0, 0], [-1, -2, -1]]]]))
afficher_grille(up_sobel(images))

In [ ]:
down_sobel =  nn.Conv2d(1,1,3, bias=False)
down_sobel.weight = nn.Parameter(torch.Tensor([[[[-1,-2,-1], [0, 0, 0], [1,2,1]]]]))
afficher_grille(down_sobel(images))

In [ ]:
afficher_grille(up_sobel(down_sobel(images)))

In [ ]:
afficher_grille(left_sobel(right_sobel(up_sobel(down_sobel(images)))))

### «Max Pooling»

In [ ]:
nn.MaxPool2d?

In [ ]:
mp = nn.MaxPool2d(2)
mp(images).shape

In [ ]:
afficher_grille(mp(images))

In [ ]:
afficher_grille(mp(up_sobel(images)))

In [ ]:
afficher_grille(down_sobel(mp(up_sobel(images))))